#BERTmoji

### imports

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from pathlib import Path
import time
import random

# Check if we are running on a CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We need to install the transformers library first, if we use Google Colab.

In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 901kB 41.0MB/s 
     |████████████████████████████████| 3.3MB 40.9MB/s 


From transformers, we need only the model itself, which is `DistilBertForSequenceClassification`; special optimizer that works with it, which is `AdamW`; and a pretrained tokenizer `DistilBertTokenizer` to feed our data correctly into the model. 

For the sake of saving space and performance, we use `DistilBert` in this notebook.

In [3]:
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig
from transformers import DistilBertTokenizer
from transformers import get_linear_schedule_with_warmup

In [4]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

drive.mount('/content/drive')
#folds 
#dataas = "content/drive/My Drive/NLP/Data"

Mounted at /content/drive


## Emoji def pred data

In [58]:
import json
def make_uchr(code: str):
  return chr(int(code.lstrip("U+").zfill(8), 16))

lines_def = []
emojis = []
def_path = "/content/drive/My Drive/NLP/emojis.json"


with open(file = def_path) as f:
  def_data = pd.DataFrame.from_dict(json.load(f))
def_data = def_data.drop(["category","name", "senses"], axis=1)
def_data["shortcode"] = def_data["shortcode"].apply(str)
def_data["unicode"] = def_data["unicode"].apply(str)
def_data = def_data.loc[def_data["shortcode"] != "None"]

def_data["unicode"].index
for i,code in zip(def_data["unicode"].index,def_data["unicode"]):
  if len(code.split()) == 1:
    def_data["unicode"][i] = make_uchr(code)
#print(def_data["unicode"].values)
def_data

,keywords,definition,unicode,shortcode
5,[keycap],Keycap Digit Zero was approved as part of Unic...,U+0030 U+FE0F U+20E3,:zero:
140,"[m, circle]","The letter M in a circle, commonly used in to ...",Ⓜ,:m:
196,"[japanese, ?, “acceptable”, ideograph]",Japanese “acceptable” Button was approved as p...,🉑,:accept:
197,"[japanese, ?, “bargain”, ideograph]",Denotes a “good bargain” in Japanese. Japanese...,🉐,:ideograph_advantage:
201,"[japanese, ?, “open for business”, ideograph]",Means “work” in Japanese. This Emoji used to ...,🈺,:u55b6:
...,...,...,...,...
2383,"[sun, mountain, sunrise, morning]","The sun rising over a mountain range, early in...",🌄,:sunrise_over_mountains:
2384,"[night, star]","A night sky with stars, depicted on major plat...",🌃,:night_with_stars:
2385,"[umbrella, rain, clothing]","An umbrella that is closed, likely as a result...",🌂,:closed_umbrella:
2386,[fog],Foggy weather emoji. Shows fog covering a view...,🌁,:foggy:


In [ ]:
def_data.reset_index()

In [67]:
def_data.iloc[0]

keywords                                               [keycap]
definition    Keycap Digit Zero was approved as part of Unic...
unicode                                    U+0030 U+FE0F U+20E3
shortcode                                                :zero:
Name: 5, dtype: object

In [68]:
MC_20 = ['❤','😂', '👍', '🙏', '🙌', '😘', '😍', '😊', '🔥', '👏', '👌', '💪', '👊', '😉', '🎉', '😎',"😁", "💯", "😜", "👀"]
eval = pd.DataFrame([def_data.iloc[i,] for i in range(len(def_data["unicode"])) if def_data.iloc[i,]["unicode"] in MC_20 ])

In [75]:
eval.columns = ["key", "Line", "Emoji", "sc"]
eval

,key,Line,Emoji,sc
893,"[hand, gesture, pray, bow, please, thanks, fol...","Two hands placed firmly together, meaning plea...",🙏,:pray:
897,"[hooray, hand, gesture, raised, celebration]","Two hands raised in the air, celebrating succe...",🙌,:raised_hands:
962,"[eye, wink, face, tongue, joke]","A face showing a stuck-out tongue, winking at ...",😜,:stuck_out_tongue_winking_eye:
967,"[kiss, face]",An emoji face blowing a kiss; but officially c...,😘,:kissing_heart:
977,"[eyewear, smile, eye, sun, glasses, sunglasses...",A face smiling and wearing dark sunglasses tha...,😎,:sunglasses:
979,"[eye, smile, love, face]","A face with hearts instead of eyes, or Heart E...",😍,:heart_eyes:
982,"[eye, smile, face, blush]","A smiling face, with smiling eyes and rosy che...",😊,:blush:
983,"[wink, face]",A classic winky emoji; winking and smiling. Us...,😉,:wink:
991,"[laugh, joy, face, tear]",A laughing emoji which at small sizes is often...,😂,:joy:
992,"[eye, smile, grin, face]",A version of the grinning face showing smiling...,😁,:grin:


# Data preps. for Twitter


In [244]:
meta = pd.read_csv("/content/drive/My Drive/NLP/Data/meta/metadata.txt",sep="\t")
meta
folds = meta.loc[0:20,]
folds 
MC_20 = ['❤','😂', '👍', '🙏', '🙌', '😘', '😍', '😊', '🔥', '👏', '👌', '💪', '👊', '😉', '🎉', '😎',"😁", "💯", "😜", "👀"]
folds = pd.DataFrame([meta.loc[i,] for i in range(len(meta[" emoji "])) if meta.loc[i,][" emoji "] in MC_20 ])
sum(folds[" len "])

308616

In [245]:
labs = {}
for i,fold in enumerate(folds[" emoji "]):
  labs[fold]=i
labs

{'❤': 4,
 '🎉': 11,
 '👀': 16,
 '👊': 1,
 '👌': 13,
 '👍': 15,
 '👏': 6,
 '💪': 0,
 '💯': 12,
 '🔥': 7,
 '😁': 9,
 '😂': 19,
 '😉': 10,
 '😊': 17,
 '😍': 18,
 '😎': 8,
 '😘': 14,
 '😜': 5,
 '🙌': 2,
 '🙏': 3}

In [246]:
dataa = {}

for i,fold in enumerate(folds["index "]):
  p = pd.read_csv("/content/drive/My Drive/NLP/Data" +"/"+ str(fold)+".csv")
  perc = np.random.rand(p.shape[0])
  perc = perc > 0.9
  p["test"] = perc
  dataa[i] = p

In [264]:
dataa[5]

,Unnamed: 0,Line,Emoji,test
0,327,john barrowman should never be allowed to forg...,😜,True
1,348,i been let that hurt go idc no more,😜,False
2,458,follow spreeeee fashion photograph music marve...,😜,False
3,517,i'm not sure we want to find out,😜,False
4,523,yellin on waianaes side wearing a saint louis ...,😜,False
...,...,...,...,...
5058,928004,should of came to dunkin,😜,False
5059,928201,it was a jab at muslims ham,😜,False
5060,928416,waiting for election results be like,😜,True
5061,928432,i've gotten tamra a few times,😜,True


# Data preps for MC20


In [208]:
MC_20 = ['❤','😂', '👍', '🙏', '🙌', '😘', '😍', '😊', '🔥', '👏', '👌', '💪', '👊', '😉', '🎉', '😎',"😁", "💯", "😜", "👀"]
data_path = '/content/drive/MyDrive/NLP/NLP project/datasets/MC_20_dev.csv'

data_set = pd.read_csv(data_path)
data_set.columns = ['Line', 'Emoji']
 # take 10000 i.e 
#data_set.loc[data_set['lable']=='😂']


In [209]:
dataa = {}

for i,lab in enumerate(MC_20):
  p = data_set.loc[data_set['Emoji']== lab]
  perc = np.random.rand(p.shape[0])
  perc = perc > 0.9
  p["test"] = perc
  dataa[i] = p

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [220]:
dataa[16]

,Line,Emoji,test
5,@benmckenna @SkyNews Thanks Ben. Yes. *#*^+ #=...,😁,False
7,आजकल '2000 के नोट में चिप लगी है जिससे इनकम टै...,😁,False
13,“@Ciinnnddddyyyy: @mgracex0 can you PLEASE tw...,😁,False
68,This how 24 feel https://t.co/D8EY7VKCXC,😁,False
73,RT @dbirdw: @FrankWaln Can’t wait,😁,False
...,...,...,...
399813,Selection Sunday,😁,False
399838,"Lost to fluminense, but the boys dug deep to b...",😁,False
399862,“@boytroublez: @MrSuPeRsTar_30 ️ I still love ...,😁,False
399902,RT @pittman_nairobi: @BurtonABC7 @ABC7Chicago ...,😁,False


In [124]:
import re
test = dataa[0]["Line"][6]
print(test)
s = re.sub(r'http\S+', '', test)
s = re.sub(r'@\S+', '', s)
s = re.sub(r'[^\w\s]','',s)
s = re.sub(r'\w+_\S+',"", s)
print(s)

RT @SarahKSilverman:  but also ️️️️️️️️ https://t.co/kK1TbVlx4D
RT   but also  


## BERT himself

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case=True, padding_side="right")

In [ ]:
print(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today."))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today.")))
tokenizer.encode_plus("This is the BERT tokenizer that we're going to use today.")

['this', 'is', 'the', 'bert', 'token', '##izer', 'that', 'we', "'", 're', 'going', 'to', 'use', 'today', '.']
[2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012]


{'input_ids': [101, 2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
PAD = '[PAD]'
PAD_ID = 0
DATA_PATH = Path('data') / 'aclImdb'

batch_size = 16
validation_split = .1
shuffle_dataset = True
random_seed = 42

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

In [248]:
class EmojiDataSet(Dataset):
    def __init__(self, folders, pretrain_tokenizer, data, test=False):
        self.tokenizer = pretrain_tokenizer
        self.label_vocab = folders
        self.max_len = 128
        self.sign = False
        self.in_data= data

        if test:
            self.sign = True
        else:
            self.sign = False
            
        self.data = []
        
        self.load()
        
    def load(self):
        for label in self.label_vocab:
          p = self.in_data[self.label_vocab[label]].dropna()
          p = p.loc[p["test"]==self.sign]
          perc2 = np.random.rand(p.shape[0])
          perc2 = perc2 < 0.66 # for twitter
          #perc2 = perc2 < 0.5 # for MC20
          p["inc"] = perc2
          p = p.loc[p["inc"]==True]
          print(f'Reading {label} sentences...')
          for line in p["Line"]:
            #s = re.sub(r'http\S+', '', line)# only for MC20
            #s = re.sub(r'@\S+', '', s)# only for MC20
            #s = re.sub(r'[^\w\s]','',s)# only for MC20
            #line = re.sub(r'\w+_\S+',"", s)# only for MC20
            text = self.tokenizer.encode(line, add_special_tokens = True, max_length=self.max_len, padding= "max_length", truncation = True, return_tensors = "pt")
            attention_mask = text > 0
            attention_mask = attention_mask.squeeze()
            torch_label = torch.tensor(self.label_vocab[label], dtype = torch.long)
              # append text, attention text and torch_label
            self.data.append((text.squeeze(), attention_mask, torch_label))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1], self.data[idx][2]

In [ ]:
train_data = EmojiDataSet(folders = labs,pretrain_tokenizer=tokenizer, data=dataa)

In [249]:
test_data = EmojiDataSet(folders = labs,pretrain_tokenizer=tokenizer, data=dataa, test=True)

Reading 💪 sentences...
Reading 👊 sentences...
Reading 🙌 sentences...
Reading 🙏 sentences...
Reading ❤ sentences...
Reading 😜 sentences...
Reading 👏 sentences...
Reading 🔥 sentences...
Reading 😎 sentences...
Reading 😁 sentences...
Reading 😉 sentences...
Reading 🎉 sentences...
Reading 💯 sentences...
Reading 👌 sentences...
Reading 😘 sentences...
Reading 👍 sentences...
Reading 👀 sentences...
Reading 😊 sentences...
Reading 😍 sentences...
Reading 😂 sentences...


In [250]:
len(test_data)

20621

In [ ]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [251]:
#train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
#validation_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels= len(train_data.label_vocab), output_attentions = False, output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The DistilBERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The DistilBERT model has 104 different named parameters.

==== Embedding Layer ====

distilbert.embeddings.word_embeddings.weight            (30522, 768)
distilbert.embeddings.position_embeddings.weight          (512, 768)
distilbert.embeddings.LayerNorm.weight                        (768,)
distilbert.embeddings.LayerNorm.bias                          (768,)
distilbert.transformer.layer.0.attention.q_lin.weight     (768, 768)

==== First Transformer ====

distilbert.transformer.layer.0.attention.q_lin.bias           (768,)
distilbert.transformer.layer.0.attention.k_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.k_lin.bias           (768,)
distilbert.transformer.layer.0.attention.v_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.v_lin.bias           (768,)
distilbert.transformer.layer.0.attention.out_lin.weight   (768, 768)
distilbert.transformer.layer.0.attention.out_lin.bias         (768,)
distilbert.transformer.layer.0.sa_layer_norm.weight      

In [ ]:
lrs = [5e-5, 4e-5, 3e-5, 2e-5]
optimizer = AdamW(model.parameters(), lr = lrs[0], eps = 1e-8)

batch_sizes = [16,32]
batch_size = 16
epochs = 3
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 5000,
                                            num_training_steps = total_steps)

## Measures

In [16]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
def MRR(preds, labels, labs):
  n = len(labs)
  ranks = np.argsort(preds.T, axis=0)[-n:][::-1]
  x = np.tile(labels, (n, 1))
  #print(preds, x, ranks)
  return np.sum(np.sum(ranks == x, axis=1)*(1/(np.arange(n)+1)))/len(labels)

In [159]:
from sklearn.metrics import precision_recall_fscore_support
def multi_label_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = np.argmax(preds, axis=1).flatten()
    #print(rounded_preds, y)
    #precision, recall, f_score, n = precision_recall_fscore_support(y, rounded_preds,labels = np.arange(20), average=None) # macro
    #precision, recall, f_score, n = precision_recall_fscore_support(y, rounded_preds, average="macro") # macro
    
    #confusion_vector = rounded_preds / y
    
    #true_positives = np.sum(confusion_vector == 1)
    #false_positives = np.sum(np.isinf(confusion_vector))
    #false_negatives = np.sum(confusion_vector == 0)
    #true_negatives = np.sum(np.isnan(confusion_vector))

    accuracy = (np.sum(rounded_preds == y)) / (len(y))
    #precision = (true_positives) / (true_positives+false_positives)
    #recall = (true_positives) / (true_positives+false_negatives)
    #f_score = 2*(precision*recall)/(precision+recall)
    #return accuracy, precision, recall, f_score, n
    return accuracy, rounded_preds, y

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training and testing

In [ ]:
# Taken from this tutorial: https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb
# The code was modified

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed_mins, elapsed_secs = epoch_time(t0, time.time())
            
            # Report progress.
            print(f'  Batch {step:>5,}  of  {len(train_loader):>5,}.    Elapsed: {elapsed_mins:}m {elapsed_secs:}s.')

        # Unpack this training batch from our dataloader. 
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, attention_mask= b_input_mask, labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print(f"  Average training loss: {avg_train_loss:.4f}")
    print("  Training epcoh took: {:}m {:}s".format(*epoch_time(t0, time.time())))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy, eval_pre, eval_rec, eval_f1 = 0, 0, 0, 0, 0
    mrr_eval = 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
            outputs = model(b_input_ids, attention_mask= b_input_mask)


        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()
        #print(label_ids)
        #print( np.argmax(logits, axis=1).flatten())
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = multi_label_accuracy(logits, label_ids)
        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        tmp_eval_mrr = MRR(logits, label_ids, labs)
        # Accumulate the total accuracy.
        #eval_accuracy += tmp_eval_accuracy
        eval_accuracy += tmp_eval_accuracy[0]
        eval_pre += tmp_eval_accuracy[1]
        eval_rec += tmp_eval_accuracy[2]
        eval_f1 += tmp_eval_accuracy[3]
        mrr_eval += tmp_eval_mrr
        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.6f}".format(eval_accuracy/nb_eval_steps))
    print("  Precision: {0:.2f}".format(eval_pre/nb_eval_steps))
    print("  Recall: {0:.2f}".format(eval_rec/nb_eval_steps))
    print("  F1_score: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  MRR_score: {0:.3f}".format(mrr_eval/nb_eval_steps))
    print("  Validation took: {:}m {:}s".format(*epoch_time(t0, time.time())))

print("")
print("Training complete!")

In [252]:

print("")
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
test_loss = 0
test_accuracy, test_pre, test_rec, test_f1 = np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)
counter_y = []
counter_pred = []
test_mrr = 0
nb_test_steps, nb_test_examples = 0, 0

# Evaluate data for one epoch
for batch in test_loader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, b_input_mask)


    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to("cpu").numpy()
    #print(label_ids)
    #print( np.argmax(logits, axis=1).flatten())
    # Calculate the accuracy for this batch of test sentences.
    #tmp_test_accuracy = flat_accuracy(logits, label_ids)

    tmp_test_accuracy = multi_label_accuracy(logits, label_ids)
    tmp_eval_mrr = MRR(logits, label_ids, labs)
    # Accumulate the total accuracy.
    #print(tmp_test_accuracy)
    test_accuracy += tmp_test_accuracy[0]
    #test_pre += tmp_test_accuracy[1]
    #test_rec += tmp_test_accuracy[2]
    #test_f1 += tmp_test_accuracy[3]
    test_mrr += tmp_eval_mrr
    counter_y += tmp_test_accuracy[2].tolist()

    counter_pred += tmp_test_accuracy[1].tolist()
    #print(tmp_test_accuracy)
    # Track the number of batches
    nb_test_steps += 1

# Report the final accuracy for this test run.
print("  Accuracy: {0:.2f}".format(test_accuracy/nb_test_steps))
print("  Precision: {0:.2f}".format(test_pre/nb_test_steps))
print("  Recall: {0:.2f}".format(test_rec/nb_test_steps))
print("  F1_score: {0:.2f}".format(test_f1/nb_test_steps))
print("  MRR_score: {0:.3f}".format(test_mrr/nb_test_steps))
print("  Testing took: {:}m {:}s".format(*epoch_time(t0, time.time())))


Running Testing...


TypeError: ignored

In [253]:
test_mrr/nb_test_steps

0.6886797328170433

In [259]:
precision, recall, f_score, n = precision_recall_fscore_support(counter_y, counter_pred,labels = np.arange(20), average=None)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [263]:
print(sum(np.array(counter_pred) == np.array(counter_y))/len(counter_y))

0.5647640754570583


In [257]:
inv_voc

{0: '💪',
 1: '👊',
 2: '🙌',
 3: '🙏',
 4: '❤',
 5: '😜',
 6: '👏',
 7: '🔥',
 8: '😎',
 9: '😁',
 10: '😉',
 11: '🎉',
 12: '💯',
 13: '👌',
 14: '😘',
 15: '👍',
 16: '👀',
 17: '😊',
 18: '😍',
 19: '😂'}

In [260]:
res = pd.DataFrame()
res["Emoji"] = inv_voc.values()
res["precision"] = precision
res["recall"] = recall
res["f_score"] = f_score

In [262]:
res

,Emoji,precision,recall,f_score
0,💪,0.454545,0.246914,0.320000
1,👊,0.371134,0.169014,0.232258
2,🙌,0.407407,0.053140,0.094017
3,🙏,0.491736,0.445693,0.467583
4,❤,0.626168,0.241877,0.348958
5,😜,0.000000,0.000000,0.000000
6,👏,0.457143,0.268156,0.338028
7,🔥,0.651515,0.552463,0.597914
8,😎,0.417021,0.168966,0.240491
9,😁,0.368421,0.012704,0.024561


## Manual testing

In [242]:
def predict_emoji(model, sentence, voc):
    model.eval()
    text = tokenizer.encode(sentence, add_special_tokens = True, max_length=128, padding= "max_length", truncation = True, return_tensors = "pt")
    attention_mask = text > 0
    attention_mask = attention_mask.squeeze().to(device)
    text = text.to(device)
    bind = (text, attention_mask)
    #print(text, attention_mask)
    with torch.no_grad():
      outputs = model(bind[0], bind[1])[0]
    prediction, indices = torch.topk(torch.sigmoid(outputs), 1)
    indices = indices.detach().cpu().numpy()
    mojis = [voc[pred]for pred in indices[0]]
    return mojis

    


In [ ]:
inv_voc = {}
for m in labs:
  inv_voc[labs[m]] = m
inv_voc

In [238]:
eval = eval.reset_index(drop=True)


In [243]:
for i in range(eval.shape[0]):
  print(eval["Line"][i].split(".")[0])
  print(predict_emoji(model, eval["Line"][i].split(".")[0], inv_voc), eval["Emoji"][i])

Two hands placed firmly together, meaning please or thank you in Japanese culture
['😊'] 🙏
Two hands raised in the air, celebrating success or another joyous event
['🎉'] 🙌
A face showing a stuck-out tongue, winking at the same time
['😂'] 😜
An emoji face blowing a kiss; but officially called “Face Throwing A Kiss”
['😂'] 😘
A face smiling and wearing dark sunglasses that is used to denote a sense of cool
['😎'] 😎
A face with hearts instead of eyes, or Heart Eyes Emoji as it is generally known
['😍'] 😍
A smiling face, with smiling eyes and rosy cheeks
['😊'] 😊
A classic winky emoji; winking and smiling
['😊'] 😉
A laughing emoji which at small sizes is often mistaken for being tears of sadness
['😂'] 😂
A version of the grinning face showing smiling eyes
['😂'] 😁
A classic red love heart emoji, used to express love
['😍'] ❤
A small flame, mostly yellow but red at the top
['🔥'] 🔥
100 emoji: the number one-hundred, written in red, underlined twice for emphasis
['👌'] 💯
An arm flexing to show its biceps

## Saving and loading

In [ ]:

import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

#output_dir = '/content/drive/My Drive/NLP/Data/bert_mc_20'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
#torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/drive/My Drive/NLP/Data/bert_mc_20


('/content/drive/My Drive/NLP/Data/bert_mc_20/tokenizer_config.json',
 '/content/drive/My Drive/NLP/Data/bert_mc_20/special_tokens_map.json',
 '/content/drive/My Drive/NLP/Data/bert_mc_20/vocab.txt',
 '/content/drive/My Drive/NLP/Data/bert_mc_20/added_tokens.json')

In [221]:
# Load a trained model and vocabulary that you have fine-tuned

output_dir = '/content/drive/My Drive/NLP/Data/bert_twitter'
model = DistilBertForSequenceClassification.from_pretrained(output_dir)
tokenizer = DistilBertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       